In [11]:
import evalml
import pandas as pd

In [ ]:

X, y = evalml.demos.load_breast_cancer()
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='binary')
X_train.head()

         Number of Features
Numeric                  30

Number of training examples: 569
Targets
benign       62.74%
malignant    37.26%
Name: count, dtype: object
<class 'pandas.core.frame.DataFrame'>


/Users/jemimagoodall/personal_projects/auto_ML_playground/.venv/lib/python3.9/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/jemimagoodall/personal_projects/auto_ML_playground/.venv/lib/python3.9/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/jemimagoodall/personal_projects/auto_ML_playground/.venv/lib/python3.9/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
381,11.04,14.93,70.67,372.7,0.07987,0.07079,0.03546,0.020740,0.2003,0.06246,...,12.090,20.83,79.73,447.1,0.1095,0.1982,0.15530,0.06754,0.3202,0.07287
144,10.75,14.97,68.26,355.3,0.07793,0.05139,0.02251,0.007875,0.1399,0.05688,...,11.950,20.72,77.79,441.2,0.1076,0.1223,0.09755,0.03413,0.2300,0.06769
136,11.71,16.67,74.72,423.6,0.10510,0.06095,0.03592,0.026000,0.1339,0.05945,...,13.330,25.48,86.16,546.7,0.1271,0.1028,0.10460,0.06968,0.1712,0.07343
116,8.95,15.76,58.74,245.2,0.09462,0.12430,0.09263,0.023080,0.1305,0.07163,...,9.414,17.07,63.34,270.0,0.1179,0.1879,0.15440,0.03846,0.1652,0.07722
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.152000,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.1650,0.8681,0.93870,0.26500,0.4087,0.12400


In [ ]:
# inspect what other problem types there are:
evalml.problem_types.ProblemTypes.all_problem_types

In [41]:
evalml.objectives.utils.get_optimization_objectives(problem_type='binary')

In [42]:
from evalml.automl import AutoMLSearch
auto_ml = AutoMLSearch(
    X_train=X_train, y_train=y_train, 
    problem_type='binary', 
    objective="f1",
    additional_objectives=['precision', 'AUC'],
    max_batches=1, 
    optimize_thresholds=True)

auto_ml.search()

{1: {'Random Forest Classifier w/ Label Encoder + Imputer + RF Classifier Select From Model': 1.351731300354004,
  'Total time of batch': 1.466521978378296}}

In [43]:
top_pipeline_id = auto_ml.rankings.iloc[0]['id']
print(f"Id of top ranking pipeline: {top_pipeline_id}")
auto_ml.rankings

Id of top ranking pipeline: 1


,id,pipeline_name,search_order,ranking_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Random Forest Classifier w/ Label Encoder + Im...,1,0.927382,0.927382,0.033263,92.738192,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,0,Mode Baseline Binary Classification Pipeline,0,0.000000,0.000000,0.000000,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [44]:
best_pipeline = auto_ml.best_pipeline
best_pipeline

pipeline = BinaryClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Imputer': ['Imputer', 'X', 'Label Encoder.y'], 'RF Classifier Select From Model': ['RF Classifier Select From Model', 'Imputer.x', 'Label Encoder.y'], 'Random Forest Classifier': ['Random Forest Classifier', 'RF Classifier Select From Model.x', 'Label Encoder.y']}, parameters={'Label Encoder':{'positive_label': None}, 'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'boolean_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None, 'boolean_fill_value': None}, 'RF Classifier Select From Model':{'number_features': None, 'n_estimators': 10, 'max_depth': None, 'percent_features': 0.5, 'threshold': 'median', 'n_jobs': -1}, 'Random Forest Classifier':{'n_estimators': 100, 'max_depth': 6, 'n_jobs': -1}}, random_seed=0)

In [45]:
auto_ml.describe_pipeline(top_pipeline_id)


*****************************************************************************************
* Random Forest Classifier w/ Label Encoder + Imputer + RF Classifier Select From Model *
*****************************************************************************************

Problem Type: binary
Model Family: Random Forest

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * boolean_impute_strategy : most_frequent
	 * categorical_fill_value : None
	 * numeric_fill_value : None
	 * boolean_fill_value : None
3. RF Classifier Select From Model
	 * number_features : None
	 * n_estimators : 10
	 * max_depth : None
	 * percent_features : 0.5
	 * threshold : median
	 * n_jobs : -1
4. Random Forest Classifier
	 * n_estimators : 100
	 * max_depth : 6
	 * n_jobs : -1

Training
Training for binary problems.
Objective to optimize binary classification pipeline thresholds for: <evalml.objectives.stan

In [46]:
# Eval best pipeline on test data:
best_pipeline.score(X_test, y_test, objectives=['f1', 'precision', 'recall', 'AUC'])

OrderedDict([('F1', 0.9090909090909091),
             ('Precision', 0.8695652173913043),
             ('Recall', 0.9523809523809523),
             ('AUC', 0.9857804232804233)])

In [49]:
# save and load models:
best_pipeline.save("model.pkl")
loaded_model = auto_ml.load("model.pkl")
loaded_model.predict_proba(X_test)

,benign,malignant
477,0.999558,0.000442
558,0.818316,0.181684
537,0.859695,0.140305
322,0.978052,0.021948
474,0.958361,0.041639
...,...,...
364,0.999821,0.000179
518,0.882032,0.117968
354,0.990000,0.010000
23,0.000000,1.000000
